In [1]:
import pandas as pd
import numpy as np
import warnings
from src.preprocessing import preproc
from catboost import Pool
from src.aggregate import new_features

warnings.filterwarnings("ignore")

In [42]:
data = pd.read_excel("Тест.xlsx")

In [43]:
data

,№ п/п,obj_prg,obj_subprg,obj_key,obj_shortName,Кодзадачи,НазваниеЗадачи,ПроцентЗавершенияЗадачи,ДатаНачалаЗадачи,ДатаОкончанияЗадачи,ДатаначалаБП0,ДатаокончанияБП0,Статуспоэкспертизе,Экспертиза
0,36,Образование,Дошкольные учреждения,020-0684,"ДОУ на 125, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-02-01,2020-11-03,2021-12-29,NaN,NaN
1,35,Образование,Дошкольные учреждения,019-0589,"ДОУ на 225, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2020-11-03,2022-05-16,2020-11-03,2021-12-29,NaN,NaN
2,61,Образование,Общеобразовательные учреждения,019-0594,"Школа на 800, ТПУ ""Мневники""",1,Предпроектные работы,0.0,2021-05-04,2021-12-15,2021-05-04,2021-12-15,NaN,NaN
3,89,Культура,Культурные центры,021-0458,"КСЦ ""Печатники"", Полбина",1,Предпроектные работы,100.0,2021-10-12,2023-05-29,2021-10-12,2023-05-12,NaN,NaN
4,34,Образование,Дошкольные учреждения,017-0520,"ДОУ на 350, ул. 6-я Радиальная и ул. Дуговая",1,Предпроектные работы,100.0,2018-11-01,2022-02-15,2018-11-01,2022-02-15,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64659,21,Здравоохранение,Поликлиники,023-0135,"ДП на 320, ул. Зеленоградская, 2 этап. Ликвид...",NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
64660,28,Здравоохранение,Поликлиники,021-0049,Пристройка к поликлинике ул. Кременчугская,NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
64661,29,Здравоохранение,Поликлиники,021-0050,"ДВП на 560, Проектируемый проезд 3538",NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN
64662,31,Здравоохранение,Поликлиники,022-0707,"ДВП на 320, Онежская",NaN,NaN,NaN,NaT,NaT,NaT,NaT,NaN,NaN


In [44]:
# data = pd.read_csv("sample.csv")
cols = pd.read_pickle("cols.pkl")
data = data[cols]
data['date_report'] = data['ДатаНачалаЗадачи']
data['marker'] = 1
data.sort_values("date_report", inplace=True)

In [45]:
filename_data = "dataset_hackaton_ksg__v2__23062023__1710_GMT3.csv"
filename_attr = "data_mgz_attributes__24062023__1000_GMT3.csv"
df = pd.read_csv(filename_data, sep=';', index_col='Unnamed: 0')
attr = pd.read_csv(filename_attr, sep=';', index_col='Unnamed: 0')
df['marker'] = 0

In [49]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 64664 entries, 162 to 64663
Data columns (total 15 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   № п/п                    64664 non-null  int64         
 1   obj_prg                  64664 non-null  object        
 2   obj_subprg               64664 non-null  object        
 3   obj_key                  64664 non-null  object        
 4   Кодзадачи                64651 non-null  object        
 5   НазваниеЗадачи           64648 non-null  object        
 6   ПроцентЗавершенияЗадачи  64651 non-null  float64       
 7   ДатаНачалаЗадачи         64651 non-null  datetime64[ns]
 8   ДатаОкончанияЗадачи      64651 non-null  datetime64[ns]
 9   ДатаначалаБП0            50468 non-null  datetime64[ns]
 10  ДатаокончанияБП0         50496 non-null  datetime64[ns]
 11  Статуспоэкспертизе       979 non-null    float64       
 12  Экспертиза               206 n

In [50]:
df['ДатаОкончанияЗадачи'] = pd.to_datetime(df['ДатаОкончанияЗадачи'])
df['ДатаНачалаЗадачи'] = pd.to_datetime(df['ДатаНачалаЗадачи'])
df['date_report'] = pd.to_datetime(df['date_report'])

In [48]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1060165 entries, 0 to 63508
Data columns (total 15 columns):
 #   Column                   Non-Null Count    Dtype  
---  ------                   --------------    -----  
 0   № п/п                    1060165 non-null  int64  
 1   obj_prg                  1060165 non-null  object 
 2   obj_subprg               1060165 non-null  object 
 3   obj_key                  1060164 non-null  object 
 4   Кодзадачи                1059960 non-null  object 
 5   НазваниеЗадачи           1059799 non-null  object 
 6   ПроцентЗавершенияЗадачи  1059960 non-null  float64
 7   ДатаНачалаЗадачи         1059960 non-null  object 
 8   ДатаОкончанияЗадачи      1059960 non-null  object 
 9   ДатаначалаБП0            797152 non-null   object 
 10  ДатаокончанияБП0         800439 non-null   object 
 11  Статуспоэкспертизе       15781 non-null    float64
 12  Экспертиза               3421 non-null     object 
 13  date_report              1060165 non-null  o

In [53]:
data_ = pd.concat([df.reset_index(), data.reset_index()], axis=0, ignore_index=True).drop("index", axis=1)
data_.sort_values("date_report", inplace=True)

In [54]:
a = preproc(data_, attr, task_name="INFER")
a = new_features(a)

In [56]:
inference_data = a[a['marker']==1].copy()

In [57]:
final_feat = pd.read_pickle("artifacts/final_feat.pkl")
new_cat_feat = pd.read_pickle("artifacts/new_cat_feat.pkl")

In [58]:
infer_pool = Pool(data=inference_data[final_feat], cat_features=new_cat_feat)

In [60]:
models = pd.read_pickle("artifacts/models_.pkl")

In [61]:
pred = 0

for model in models:
    pred += model.predict(infer_pool)

res = np.round((pred / 4).clip(0))

In [64]:
inference_data['predict'] = res

In [ ]:
inference_data['preds'] = np.abs(inference_data['target'] - inference_data['predict'])
inference_data.to_csv("new_sample.csv", index=False)

In [65]:
import shap

In [73]:
models = pd.read_pickle("artifacts/models_.pkl")
model = models[0]

In [74]:
explainer = shap.Explainer(model)

In [ ]:
shap_values = explainer(inference_data[final_feat])

In [31]:
pd.to_pickle(shap_values, "artifacts/shap_values.pkl")